In [1]:
%cd ..

e:\Github\Safety_helmet_YOLOV11


# Loading our Libraries

In [2]:
import torch
import torchvision
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
import mlflow
import optuna
import onnx
import onnxruntime as ort

In [3]:
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch: 2.7.0+cu118
Torchvision: 0.22.0+cu118
CUDA available: True


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


# Importing our Data

In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="ppCtVMieB0hiwoXIAewl")
project = rf.workspace("marwanswork").project("safety-helmet-q3b8o-mz02g")
version = project.version(1)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...


# Identifying our parameters

In [6]:
data_path = r'safety-helmet-1/data.yaml'
epochs_num = 100
img_sz = 640

# Using Yolo Large Model

In [7]:
model = YOLO("yolo11m.pt")

In [8]:
# Start MLflow experiment
mlflow.set_experiment("SafetyHelmet_mlflow")

<Experiment: artifact_location='file:///e:/Github/Safety_helmet_YOLOV11/mlruns/230612644286972628', creation_time=1750634012814, experiment_id='230612644286972628', last_update_time=1750634012814, lifecycle_stage='active', name='SafetyHelmet_mlflow', tags={}>

In [9]:
from ultralytics.utils import SETTINGS
SETTINGS['mlflow'] = False

In [10]:
import yaml

# Load params.yaml
with open("params.yaml", "r") as file:
    params = yaml.safe_load(file)


In [11]:
def goal_trial(trial):
    
    # Hyperparameter tuning
    batch = trial.suggest_categorical("batch", params["batch"]["values"])
    imgsz = trial.suggest_categorical("imgsz", params["imgsz"]["values"])
    lr0 = trial.suggest_float("lr0", params["lr0"]["min"], params["lr0"]["max"], log=params["lr0"].get("log", False))
    dropout = trial.suggest_float("dropout", params["dropout"]["min"], params["dropout"]["max"])

    run_name = f"trial_{trial.number}"
    mlflow.start_run(run_name=run_name)

    try:
        # Log hyperparameters
        mlflow.log_params({
            "batch": batch,
            "imgsz": imgsz,
            "lr0": lr0,
            "dropout": dropout
        })

        # Load and train model
        model = YOLO("yolo11m.pt")
        results = model.train(
            data=data_path,
            epochs=10,
            imgsz=imgsz,
            batch=batch,
            optimizer="AdamW",
            lr0=lr0,
            dropout=dropout,
            project="runs",
            name=run_name,
            verbose=False
        )

        # Exporting the model and validating it
        model.export(format="onnx")
        model.val(name="eval_run", project="optuna_runs")

        # Extract & Log mAP50 metric
        map50 = results.box.map50
        mlflow.log_metric("mAP50", map50)

        mlflow.end_run()
        return map50

    except Exception as e:
        mlflow.end_run(status="FAILED")
        raise e

# Run Optuna Trials
study = optuna.create_study(direction="maximize")
study.optimize(goal_trial, n_trials=5)



[I 2025-06-23 09:43:34,557] A new study created in memory with name: no-name-3ea669ba-0761-4688-bfa2-548bde9e87c6


New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.145  Python-3.11.11 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=safety-helmet-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.1627631161679714, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008152623306558295, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=trial_06,

train: Scanning E:\Github\Safety_helmet_YOLOV11\safety-helmet-1\train\labels.cache... 1005 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1005/1005 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.1 ms, read: 461.6216.3 MB/s, size: 54.4 KB)


val: Scanning E:\Github\Safety_helmet_YOLOV11\safety-helmet-1\valid\labels.cache... 117 images, 0 backgrounds, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.65it/s]


                   all        117        445      0.947      0.933       0.97      0.631
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\trial_4
Ultralytics 8.3.145  Python-3.11.11 torch-2.7.0+cu118 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs

PyTorch: starting from 'runs\trial_4\weights\best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 6, 336) (38.6 MB)
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING requirements:  Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.

ONNX: starting export with onnx 1.17.0 

val: Scanning E:\Github\Safety_helmet_YOLOV11\safety-helmet-1\valid\labels.cache... 117 images, 0 backgrounds, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.58it/s]


                   all        117        445      0.946      0.933      0.969      0.631
Speed: 0.1ms preprocess, 2.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to optuna_runs\eval_run17


[I 2025-06-23 10:14:31,646] Trial 4 finished with value: 0.9696193699509699 and parameters: {'batch': 32, 'imgsz': 128, 'lr0': 0.0012237631940762994, 'dropout': 0.17907990742894425}. Best is trial 3 with value: 0.9792765225278424.


In [12]:
print("Best Trial Parameters:", study.best_trial.params)
print("Best Trial:", study.best_trial.number)

Best Trial Parameters: {'batch': 16, 'imgsz': 640, 'lr0': 7.801812094079804e-05, 'dropout': 0.22005437981089737}
Best Trial: 3
